# Modeling
- Ignoring data before 2006.
- Using only type=5 examples.
- Use a fixed window (more recent) of training data.

In [18]:
import pandas as pd
import numpy as np
import os 

import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [19]:
# Private DataSet path: ../input/kddbr2018dataset/kddbr-2018-dataset/dataset. This dataset is the same of competitions
#
path = '../input/'
print(os.listdir(path))

['field-22.csv', 'field-20.csv', 'sample-submission.csv', 'field-3.csv', 'soil_data.csv', 'field-18.csv', 'field-23.csv', 'field-17.csv', 'field-2.csv', 'field-27.csv', 'field-24.csv', 'field-10.csv', 'field-11.csv', 'field-26.csv', 'field-21.csv', 'test.csv', 'field-25.csv', 'field-19.csv', 'field-7.csv', 'field-15.csv', 'field-8.csv', 'field-12.csv', 'field-0.csv', 'field-1.csv', 'field-14.csv', 'train.csv', 'field-5.csv', 'field-16.csv', 'field-13.csv', 'field-9.csv', 'field-6.csv', 'field-4.csv']


## Train and test datasets

Basic data containing palm tree information

In [20]:
df_train = pd.read_csv(os.path.join(path, 'train.csv'))

# Ignoring date before 2006
df_train = df_train[df_train.harvest_year >= 2006]

# Using only examples with type=5
df_train = df_train[df_train.type == 5]

df_test  = pd.read_csv(os.path.join(path, 'test.csv'))
df_all = pd.concat([df_train, df_test], sort=False)

print(df_train.shape, df_test.shape, df_all.shape)
df_all.head()

(3332, 7) (4110, 6) (7442, 7)


,Id,field,age,type,harvest_year,harvest_month,production
24,24,0,21,5,2006,1,0.121454
25,25,0,21,5,2006,2,0.073829
26,26,0,21,5,2006,3,0.053183
27,27,0,21,5,2006,4,0.031207
28,28,0,21,5,2006,5,0.067851


In [21]:
def to_date(df):
    return pd.to_datetime((df.harvest_year*10000+df.harvest_month*100+1)\
                                  .apply(str),format='%Y%m%d')
# Add date variable 
for d in [df_train, df_test, df_all]:
    d['date'] = to_date(d)

## Field data (field_*.csv)

These files hold atmospheric data from January 2002 to December 2017, and can be used to estimate the weather conditions during the development of the plant. Notice that weather does influence the production. Using only a single month prior to harvest is probably too little data. Participants should decide how far back in the past they want to look when training models.



In [22]:
# read
df_field = pd.read_csv(path+'field-0.csv')
df_field['field'] = 0
for i in range(1, 28):
    _df_field = pd.read_csv(path+'field-{}.csv'.format(i))
    _df_field['field'] = i
    df_field = pd.concat([df_field, _df_field])

# remove duplicates
df_field = df_field.drop_duplicates()

# Group 
df_field = df_field.groupby(['month', 'year', 'field']).mean().reset_index()
print(df_field.shape)
df_field.head()

(5376, 11)


,month,year,field,temperature,dewpoint,windspeed,Soilwater_L1,Soilwater_L2,Soilwater_L3,Soilwater_L4,Precipitation
0,1,2002,0,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
1,1,2002,1,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
2,1,2002,2,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
3,1,2002,3,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
4,1,2002,4,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55


In [23]:
# df_all
df_all = pd.merge(df_all, df_field, left_on=['harvest_year', 'harvest_month','field'], 
                  right_on=['year', 'month', 'field'], how='inner').reset_index()
print(df_all.shape)
df_all.head()

(7442, 19)


,index,Id,field,age,type,harvest_year,harvest_month,production,date,month,year,temperature,dewpoint,windspeed,Soilwater_L1,Soilwater_L2,Soilwater_L3,Soilwater_L4,Precipitation
0,0,24,0,21,5,2006,1,0.121454,2006-01-01,1,2006,26.595,24.740,1.9897,0.34587,0.34462,0.33994,0.32083,341.51
1,1,2351,0,4,5,2006,1,0.018644,2006-01-01,1,2006,26.595,24.740,1.9897,0.34587,0.34462,0.33994,0.32083,341.51
2,2,4226,0,6,5,2006,1,0.100588,2006-01-01,1,2006,26.595,24.740,1.9897,0.34587,0.34462,0.33994,0.32083,341.51
3,3,25,0,21,5,2006,2,0.073829,2006-02-01,2,2006,26.204,24.626,2.1281,0.35104,0.35031,0.34861,0.34523,309.01
4,4,2435,0,4,5,2006,2,0.018644,2006-02-01,2,2006,26.204,24.626,2.1281,0.35104,0.35031,0.34861,0.34523,309.01


In [24]:
df_all.columns

Index(['index', 'Id', 'field', 'age', 'type', 'harvest_year', 'harvest_month',
       'production', 'date', 'month', 'year', 'temperature', 'dewpoint',
       'windspeed', 'Soilwater_L1', 'Soilwater_L2', 'Soilwater_L3',
       'Soilwater_L4', 'Precipitation'],
      dtype='object')

In [25]:
# Features i will duplicate with the past months
features = ['temperature', 'dewpoint', 'windspeed', 'Precipitation', 'Soilwater_L1']

# Remove redundant features.
df_all = df_all.drop(columns=['Soilwater_L2', 'Soilwater_L3', 'Soilwater_L4'])

df_group = df_all.groupby(['field', 'date']).mean().reset_index()[['field', 'date', 'production'] + features]
df_group = df_group.sort_values(['field', 'date'])
print(df_group.shape)
df_group.head()

(3750, 8)


,field,date,production,temperature,dewpoint,windspeed,Precipitation,Soilwater_L1
0,0,2006-01-01,0.080229,26.595,24.740,1.9897,341.51,0.34587
1,0,2006-02-01,0.045585,26.204,24.626,2.1281,309.01,0.35104
2,0,2006-03-01,0.059364,26.019,24.734,2.1760,452.42,0.36100
3,0,2006-04-01,0.045729,25.978,24.882,1.5105,516.34,0.36575
4,0,2006-05-01,0.043659,26.171,24.843,1.6832,349.06,0.36109


In [26]:
# Collect shift values of variables in all features time
period = 2

new_features = {}
for f in features:
    new_features[f] = []
    for i in range(1, period):
        new_feature = '{}_{}'.format(f, i)
        new_features[f].append(new_feature)
        df_group[new_feature] = df_group[f].shift(i).fillna(df_group[f].mean())

df_group = df_group.drop(features+['production'], axis=1)
df_group.head()

,field,date,temperature_1,dewpoint_1,windspeed_1,Precipitation_1,Soilwater_L1_1
0,0,2006-01-01,27.483066,23.767851,2.15631,252.966125,0.308294
1,0,2006-02-01,26.595000,24.740000,1.98970,341.510000,0.345870
2,0,2006-03-01,26.204000,24.626000,2.12810,309.010000,0.351040
3,0,2006-04-01,26.019000,24.734000,2.17600,452.420000,0.361000
4,0,2006-05-01,25.978000,24.882000,1.51050,516.340000,0.365750


In [27]:
df_all = df_all.drop(['index', 'month', 'year'], axis=1)
df_all = pd.merge(df_all, df_group, left_on=['field', 'date'], right_on=['field', 'date'], how='inner').reset_index()

print(df_all.shape)
df_all.head()

(7442, 19)


,index,Id,field,age,type,harvest_year,harvest_month,production,date,temperature,dewpoint,windspeed,Soilwater_L1,Precipitation,temperature_1,dewpoint_1,windspeed_1,Precipitation_1,Soilwater_L1_1
0,0,24,0,21,5,2006,1,0.121454,2006-01-01,26.595,24.740,1.9897,0.34587,341.51,27.483066,23.767851,2.15631,252.966125,0.308294
1,1,2351,0,4,5,2006,1,0.018644,2006-01-01,26.595,24.740,1.9897,0.34587,341.51,27.483066,23.767851,2.15631,252.966125,0.308294
2,2,4226,0,6,5,2006,1,0.100588,2006-01-01,26.595,24.740,1.9897,0.34587,341.51,27.483066,23.767851,2.15631,252.966125,0.308294
3,3,25,0,21,5,2006,2,0.073829,2006-02-01,26.204,24.626,2.1281,0.35104,309.01,26.595000,24.740000,1.98970,341.510000,0.345870
4,4,2435,0,4,5,2006,2,0.018644,2006-02-01,26.204,24.626,2.1281,0.35104,309.01,26.595000,24.740000,1.98970,341.510000,0.345870


## Soil Data (soil_data.csv)

Information about the soil on each field.

In [28]:
df_soil = pd.read_csv(path+'soil_data.csv')
print(df_soil.shape)
df_soil.head()

(28, 73)


,field,BDRICM_BDRICM_M,BDRLOG_BDRLOG_M,BDTICM_BDTICM_M,BLDFIE_sl1,BLDFIE_sl2,BLDFIE_sl3,BLDFIE_sl4,BLDFIE_sl5,BLDFIE_sl6,...,SLTPPT_sl5,SLTPPT_sl6,SLTPPT_sl7,SNDPPT_sl1,SNDPPT_sl2,SNDPPT_sl3,SNDPPT_sl4,SNDPPT_sl5,SNDPPT_sl6,SNDPPT_sl7
0,4,200,7,6973,1345,1308,1361,1413,1486,1503,...,21,19,20,47,48,47,42,40,40,39
1,3,200,9,7272,1297,1287,1323,1428,1492,1508,...,23,22,22,44,45,43,40,36,37,36
2,2,200,7,7281,1266,1249,1310,1387,1463,1491,...,21,21,22,46,46,45,40,39,39,39
3,1,200,6,7457,1297,1277,1345,1409,1480,1506,...,21,21,21,46,47,46,42,40,40,40
4,7,200,8,6771,1305,1289,1333,1438,1497,1510,...,22,21,22,44,45,44,40,38,38,37


In [29]:
# Join datasets
df_all_soil = pd.merge(df_all, df_soil, on='field', how='inner')
print(df_all_soil.shape)
df_all_soil.head()

(7442, 91)


,index,Id,field,age,type,harvest_year,harvest_month,production,date,temperature,...,SLTPPT_sl5,SLTPPT_sl6,SLTPPT_sl7,SNDPPT_sl1,SNDPPT_sl2,SNDPPT_sl3,SNDPPT_sl4,SNDPPT_sl5,SNDPPT_sl6,SNDPPT_sl7
0,0,24,0,21,5,2006,1,0.121454,2006-01-01,26.595,...,22,22,23,44,45,44,39,38,37,36
1,1,2351,0,4,5,2006,1,0.018644,2006-01-01,26.595,...,22,22,23,44,45,44,39,38,37,36
2,2,4226,0,6,5,2006,1,0.100588,2006-01-01,26.595,...,22,22,23,44,45,44,39,38,37,36
3,3,25,0,21,5,2006,2,0.073829,2006-02-01,26.204,...,22,22,23,44,45,44,39,38,37,36
4,4,2435,0,4,5,2006,2,0.018644,2006-02-01,26.204,...,22,22,23,44,45,44,39,38,37,36


## Feature Importance Measures

Find the main features for the production target. Uses a RandomRorest to identify features


In [30]:
df_all_soil.columns

Index(['index', 'Id', 'field', 'age', 'type', 'harvest_year', 'harvest_month',
       'production', 'date', 'temperature', 'dewpoint', 'windspeed',
       'Soilwater_L1', 'Precipitation', 'temperature_1', 'dewpoint_1',
       'windspeed_1', 'Precipitation_1', 'Soilwater_L1_1', 'BDRICM_BDRICM_M',
       'BDRLOG_BDRLOG_M', 'BDTICM_BDTICM_M', 'BLDFIE_sl1', 'BLDFIE_sl2',
       'BLDFIE_sl3', 'BLDFIE_sl4', 'BLDFIE_sl5', 'BLDFIE_sl6', 'BLDFIE_sl7',
       'CECSOL_sl1', 'CECSOL_sl2', 'CECSOL_sl3', 'CECSOL_sl4', 'CECSOL_sl5',
       'CECSOL_sl6', 'CECSOL_sl7', 'CLYPPT_sl1', 'CLYPPT_sl2', 'CLYPPT_sl3',
       'CLYPPT_sl4', 'CLYPPT_sl5', 'CLYPPT_sl6', 'CLYPPT_sl7', 'CRFVOL_sl1',
       'CRFVOL_sl2', 'CRFVOL_sl3', 'CRFVOL_sl4', 'CRFVOL_sl5', 'CRFVOL_sl6',
       'CRFVOL_sl7', 'OCSTHA_sd1', 'OCSTHA_sd2', 'OCSTHA_sd3', 'OCSTHA_sd4',
       'OCSTHA_sd5', 'OCSTHA_sd6', 'ORCDRC_sl1', 'ORCDRC_sl2', 'ORCDRC_sl3',
       'ORCDRC_sl4', 'ORCDRC_sl5', 'ORCDRC_sl6', 'ORCDRC_sl7', 'PHIHOX_sl1',
       'PHIHOX

In [31]:
## Import the random forest model.
from sklearn.ensemble import RandomForestRegressor

## This line instantiates the model. 
rf = RandomForestRegressor(random_state=1) 

# data
df      = df_all_soil[~df_all_soil.production.isna()]
X_train = df.drop(['production', 'date', 'Id', 'index'], axis=1)
y_train = df.production.values

## Fit the model on your training data.
rf.fit(X_train, y_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [32]:
# feature_importances
feature_importances = pd.DataFrame(rf.feature_importances_, 
                                   index = X_train.columns, 
                                   columns=['importance']).sort_values('importance', ascending=False).reset_index()
feature_importances

,index,importance
0,temperature,0.211679
1,dewpoint,0.164732
2,age,0.117088
3,Precipitation_1,0.080896
4,Precipitation,0.056180
5,Soilwater_L1,0.042393
6,dewpoint_1,0.040666
7,windspeed,0.029505
8,field,0.018820
9,temperature_1,0.018705


## Base Model 

Creation of a baseline model to finalize the competition submission pipeline. The idea is to create the most basic for future improvements.

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection

### Prepare Dataset

In [38]:
# Exclude test data.
df_train = df_all_soil[~df_all_soil.production.isna()]

# Most important features.
features = list(feature_importances['index'].values)[:15]

scores = []
num_runs = 20

# Validate using sliding window.
for val_year in range(2007, 2012):

    # Split train/val.
    df_train_train = df_train[(df_train.harvest_year < val_year) & (df_train.harvest_year >= val_year - 2)]
    df_train_test = df_train[df_train.harvest_year >= val_year]

    X_train = df_train_train[features]
    y_train = df_train_train.production.values

    X_test = df_train_test[features]
    y_test = df_train_test.production.values

    df_train_train = df_train_train.drop(['production', 'date', 'Id'], axis=1)
    df_train_test = df_train_test.drop(['production', 'date', 'Id'], axis=1)

    # Normalize features.
    scaler = StandardScaler()
    scaler.fit_transform(np.concatenate([X_train, X_test]))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    maes = []
    r2s = []
    for run in range(num_runs):
        #
        # Train
        #
        base_model = RandomForestRegressor()
        base_model.fit(X_train, y_train)

        #
        # Predict validation data.
        #
        y_hat = base_model.predict(X_test)

        #
        # Score model.
        #
        mae = sklearn.metrics.mean_absolute_error(y_test, y_hat)
        r2 = sklearn.metrics.r2_score(y_test, y_hat)
        
        maes.append(mae)
        r2s.append(r2)

    mae = np.mean(maes)
    mae_sd = np.std(maes)
    
    r2 = np.mean(r2s)
    r2_sd = np.std(r2s)
    
    print('year: {}, train: {}, test: {}, mae: {} ({}), r2: {} ({})'.format(val_year, 
                                                                            X_train.shape[0], 
                                                                            X_test.shape[0], 
                                                                            mae, mae_sd, 
                                                                            r2, r2_sd))

    scores.append(mae)
    
print('mae: {} ({})\n{}'.format(np.mean(scores), np.std(scores), scores))

year: 2007, train: 563, test: 2769, mae: 0.08500720366249671 (0.0019343669988315252), r2: 0.23420782732544043 (0.03343392325754991)
year: 2008, train: 1121, test: 2211, mae: 0.09619683383504839 (0.002838017147092082), r2: 0.12367934002517358 (0.04633154144841018)
year: 2009, train: 1081, test: 1688, mae: 0.09306849375923022 (0.002408590783809936), r2: 0.22575257098191326 (0.04449823110075566)
year: 2010, train: 1085, test: 1126, mae: 0.09934583855451001 (0.0056455624479756885), r2: 0.20706894377321644 (0.08868194819358695)
year: 2011, train: 1124, test: 564, mae: 0.10692153220031389 (0.008667799520835538), r2: 0.28134549655178215 (0.09878853352311473)
mae: 0.09610798040231985 (0.007209580377733931)
[0.08500720366249671, 0.09619683383504839, 0.09306849375923022, 0.09934583855451001, 0.10692153220031389]


### Train on all data

In [35]:
# # Exclude test data.
# df_train = df_all_soil[~df_all_soil.production.isna()]

# # Most important features.
# features = list(feature_importances['index'].values)[:15]

# X_train = df_train[features]
# y_train = df_train.production.values

# # Normalize features.
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# #
# # Train
# #
# base_model = RandomForestRegressor(random_state=1)
# base_model.fit(X_train, y_train)

### Submission

It makes the predict of the basic model and creates the sample to substrate in kaggle, finishing the complete pipeline

In [36]:
# # Get test examples.
# df_test = df_all_soil[df_all_soil.production.isna()]

# # Important features.
# X = df_test[features]

# # Normalize input features.
# X = scaler.transform(X)

# # Make prediction.
# pred = base_model.predict(X)

# # Create a submission file.
# with open('../submissions/submission-all_data-ignore_lt2006-only_type5.csv', 'w') as f:
#     f.write("Id,production\n")
#     for _id, _pred in zip(df_test.Id.values, pred):
#         f.write("{},{}\n".format(_id, _pred))